In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp importers.matrix

In [ ]:
# export
from hashlib import sha256
from integrators.data.schema import *
from integrators.imports import *
from nbdev.showdoc import show_doc
import docker

In [ ]:
class Matrix:
    
    def __init__(self, server, user, data_dir, client, my_uid, my_gid):
        self.server = server
        self.user = user
        self.dir = data_dir
        self.client = client
        self.my_uid = my_uid
        self.my_gid = my_gid
        
        assert self.server is not None
        assert self.user is not None
        assert self.dir is not None
        assert self.my_uid is not None
        assert self.my_gid is not None
        assert self.client is not None
        
    def config_matrix(self):
        os.mkdir(self.dir)
        
        self.client.containers.run(
            "matrixdotorg/synapse:latest",
            auto_remove=True,
            command="generate",
            detach=True, 
            volumes={self.dir: {'bind': '/data', 'mode': 'rw'}},
            environment=[f"UID={self.my_uid}", f"GID={self.my_gid}", f"SYNAPSE_SERVER_NAME={self.server}", "SYNAPSE_REPORT_STATS=no"],
        )
        time.sleep(5)

        find_replace = {
            "#enable_registration: false": "enable_registration: true",
            "#app_service_config_files:": "app_service_config_files:",
            "#  - app_service_1.yaml": "  - /data/registration.yaml"
        }
        with open(f"{self.dir}/homeserver.yaml") as fin:
            with open(f"{self.dir}/homeserver-bak.yaml", 'w') as fout:
                for line in fin:
                    for key in find_replace:
                        if key in line:
                            line = line.replace(key, find_replace[key])
                    fout.write(line)  
                    
        os.remove(f"{self.dir}/homeserver.yaml")
        os.rename(f"{self.dir}/homeserver-bak.yaml", f"{self.dir}/homeserver.yaml")   
        
    def run_matrix(self, networkname):
        self.client.containers.run(
            "matrixdotorg/synapse:latest", 
            detach=True, 
            network=network_name,
            restart_policy={'Name': 'on-failure'},
            ports={'8008': '8008'},
            volumes={DIR: {'bind': '/data', 'mode': 'rw'}},
            environment=[f"UID={self.my_uid}", f"GID={self.my_gid}"],
            name=self.server
        )
        time.sleep(3)
        
    def register_user(self, password):
        body = {"username": self.user, "password": password, "auth": {"type":"m.login.dummy"}}
        result = requests.post(f"http://{self.server}:8008/_matrix/client/r0/register", json=body)
        json = result.json()
        user_id = json["user_id"]
        token = json["access_token"]
        matrix_acc = Account(externalId=self.username, importJson=token, username=uid, service="matrix")
        pod_client.create(matrix_acc)
        
    def upload_configs(self, hostname, pod_client):
        matrix_importer = Importer(externalId=hostname, name="matrix")
        pod_client.create(matrix_importer)
        
        os.chdir(self.dir)
        for filename in os.listdir(os.getcwd()):
            with open(filename,"rb") as f:
                data = f.read() # read entire file as bytes
                readable_hash = sha256(data).hexdigest()
                file = File(externalId=filename, sha256=readable_hash)
                pod_client.create(file)
                pod_client.upload_file(data)
                matrix_importer.add_edge("sharedWith", file)
                f.close()     
        pod_client.create_edges(matrix_importer.get_all_edges())      

## MatrixClient

This class is responsible for the interactions with Matrix's [Client-Server API](https://matrix.org/docs/spec/client_server/r0.6.1), such as fetching the information about WhatsApp contacts, chats and most recent messages.

In [ ]:
# export
class MatrixClient:
    
    def __init__(self, url, username, token):
        self.url = url # Matrix url
        self.username = username # username of Matrix account
        self.token = token # token of Matrix account
        
        assert self.url is not None
        assert self.username is not None
        assert self.token is not None        
        
    def get_joined_rooms(self):
        """List all rooms the user joined"""
        try:
            result = requests.get(f"{self.url}/_matrix/client/r0/joined_rooms?access_token={self.token}")
            if result.status_code != 200:
                print(result, result.content)
                return False
            else:
                json = result.json()
                res = json["joined_rooms"] # only rooms with "joined" status
                return res
        except requests.exceptions.RequestException as e:
            print(e)
            return None    
    
    def get_joined_members(self, room_id):
        """List all joined members in a room"""
        try:
            result = requests.get(f"{self.url}/_matrix/client/r0/rooms/{room_id}/joined_members?access_token={self.token}")
            if result.status_code != 200:
                print(result, result.content)
                return False
            else:
                json = result.json()
                res = json["joined"] # only members with "joined" status
                return res
        except requests.exceptions.RequestException as e:
            print(e)
            return None          
        
    def send_messages(self, room_id, body):
        """Send a message to a joined room"""
        try:
            result = requests.post(f"{self.url}/_matrix/client/r0/rooms/{room_id}/send/m.room.message?access_token={self.token}", json=body)
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                json = result.json()
                event_id = json["event_id"] 
                return event_id
        except requests.exceptions.RequestException as e:
            print(e)
            return None
            
    def get_event_context(self, room_id, event_id):
        """Fetch events after a certain event"""
        try:
            # limit to context event is set to 1
            result = requests.get(f"{self.url}/_matrix/client/r0/rooms/{room_id}/context/{event_id}?limit=1&access_token={self.token}")
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                json = result.json()
                res = json["events_after"] # only show events after the specific event
                return res
        except requests.exceptions.RequestException as e:
            print(e)
            return None    
        
    def sync_events(self, next_batch):
        """Fetch all events in a room in a batch"""
        try:
            result = requests.get(f"{self.url}/_matrix/client/r0/sync?since={next_batch}&access_token={self.token}")
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                json = result.json()
                return json
        except requests.exceptions.RequestException as e:
            print(e)
            return None
        
    def get_profile(self, user_id):
        """Fetch profile of a specific user"""
        try:
            result = requests.get(f"{self.url}/_matrix/client/r0/profile/{user_id}")
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                json = result.json() 
                return json
        except requests.exceptions.RequestException as e:
            print(e)
            return None
        
    def get_room_state(self, room_id):
        """Fetch status of a room"""
        try:
            result = requests.get(f"{self.url}/_matrix/client/r0/rooms/{room_id}/state?access_token={self.token}")
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                json = result.json()
                return json
        except requests.exceptions.RequestException as e:
            print(e)
            return None
        
    def download_file(self, uri):
        """Download media files"""
        try:
            result = requests.get(f"{self.url}/_matrix/media/r0/download/{HOSTNAME}/{uri}")
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                file = result.content
                return file
        except requests.exceptions.RequestException as e:
            print(e)
            return None

In [ ]:
show_doc(MatrixClient.get_joined_rooms)

In [ ]:
show_doc(MatrixClient.get_joined_members)

In [ ]:
show_doc(MatrixClient.send_messages)

In [ ]:
show_doc(MatrixClient.get_event_context)

In [ ]:
show_doc(MatrixClient.sync_events)

In [ ]:
show_doc(MatrixClient.get_profile)

In [ ]:
show_doc(MatrixClient.get_room_state)

In [ ]:
show_doc(MatrixClient.download_file)

In [ ]:
# hide
from nbdev.export import *
notebook2script()